In [9]:
import pandas as pd
import numpy as np
import glob
import re
import json
import scipy.stats as stats
import os
import seaborn as sns
import colorcet as cc

# read in the idiosyncratic ZECMIP csv files
def read_files(files):

    def find_var(fpath):
        suffix = re.split('/', fpath)[-1]
        var = re.split('_', suffix)[0]
        return (var)

    header = [find_var(file) for file in files]
    data = pd.concat([pd.read_csv(f, header=None).iloc[:, 1]
                     for f in files], axis=1)
    data = pd.concat(
        [data, pd.read_csv(files[0], header=None).iloc[:, 0]], axis=1)
    header.append('time')
    data.columns = header
    for i in data.columns:
        data[i] = pd.to_numeric(data[i], errors='ignore')
    #remove first row of column names if present
    try:
        float(data.loc[0][0])
    except:
        data = data[1:].reset_index()
    #remove last row if Nans    
    if np.isnan(data.iloc[-1][0]):
        data = data[:-1].reset_index()
    return data

def climate_average(x, tau=20):
    half_tau = int(tau/2)
    average = np.zeros(len(x))
    for i in np.arange(len(x)):
        if (i < half_tau or len(x) - i < half_tau):
            average[i] = np.nan
        else:
            average[i] = np.mean(x[i - half_tau:i + half_tau])

    return average
    
def map_over_dict(func, d):
    new_dict = {}
    for key, value in d.items():
        if isinstance(value, dict):
            new_dict[key] = map_dict(func, value)
        else:
            new_dict[key] = func(value)
    return new_dict

In [24]:
input_prefix = os.path.realpath("datasets/ZECMIP/") 
output_prefix = os.path.realpath("results/") 

In [6]:
print("Reading in raw data from ZECMIP data download...")

names = ['ACCESS','BERN-ecs2k', 'BERN-ecs3k', 'BERN-ecs5k', 'CanESM5', 'CESM',
         'CLIMBER2','CNRM-ESM2-1', 'DCESS', 'GFDL-ESM2M', 'IAPRAS',
         'LOVECLIM', 'MESM', 'MIROC-ES2L', 'MIROC-lite', 'MPIESM',
         'NorESM2','PLASIM-GENIE','UKESM1', 'UVic']

data = {}
for model in names:
    data[model] = {}
    for Cemit in ["750", "1000", "2000"]:
        files = glob.glob(input_prefix+'/*/*/*esm-1pct-brch*'+Cemit+'*.csv')
        model_files = [item for item in files if model in item]
        if(len(model_files) > 0):
            model_data = read_files(model_files)
            data[model][Cemit] = model_data
        else:
            print("No data for "+model+" exp "+Cemit)
            
# convert data types to floats
for model in data:
    for exp in data[model].keys():
        for key in data[model][exp].keys():
            data[model][exp][key] = data[model][exp][key].astype('float')

#CanESM5 is missing the 1850 data in the 1000 GtC case
#so we copy this data in from the 2000 GtC case.
data_1850 = data["CanESM5"]["2000"].iloc[0:1]
data["CanESM5"]["1000"] = pd.concat([data_1850,  data["CanESM5"]["1000"]], ignore_index=True)

Reading in raw data from ZECMIP data download...


/tmp/ipykernel_118148/1537208724.py:29: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data[i] = pd.to_numeric(data[i], errors='ignore')
/tmp/ipykernel_118148/1537208724.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(data.loc[0][0])
/tmp/ipykernel_118148/1537208724.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.isnan(data.iloc[-1][0]):
/tmp/ipykernel_118148/1537208724.py:29: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without pa

No data for CanESM5 exp 750
No data for CESM exp 750
No data for CESM exp 2000
No data for CLIMBER2 exp 750
No data for CLIMBER2 exp 2000
No data for CNRM-ESM2-1 exp 750
No data for CNRM-ESM2-1 exp 2000


/tmp/ipykernel_118148/1537208724.py:29: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data[i] = pd.to_numeric(data[i], errors='ignore')
/tmp/ipykernel_118148/1537208724.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(data.loc[0][0])
/tmp/ipykernel_118148/1537208724.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.isnan(data.iloc[-1][0]):
/tmp/ipykernel_118148/1537208724.py:29: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without pa

No data for LOVECLIM exp 2000
No data for MPIESM exp 750
No data for MPIESM exp 2000
No data for NorESM2 exp 750


/tmp/ipykernel_118148/1537208724.py:29: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data[i] = pd.to_numeric(data[i], errors='ignore')
/tmp/ipykernel_118148/1537208724.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(data.loc[0][0])
/tmp/ipykernel_118148/1537208724.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.isnan(data.iloc[-1][0]):
/tmp/ipykernel_118148/1537208724.py:29: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without pa

No data for NorESM2 exp 2000


/tmp/ipykernel_118148/1537208724.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(data.loc[0][0])
/tmp/ipykernel_118148/1537208724.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if np.isnan(data.iloc[-1][0]):
/tmp/ipykernel_118148/1537208724.py:29: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data[i] = pd.to_numeric(data[i], errors='ignore')
/tmp/ipykernel_118148/1537208724.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer k

In [ ]:
# these values are taken from ZECMIP documentation
# as reported in ../data/'Model Metadata for ZECMIP R3.docx'
metrics = {
    "ACCESS": {
        "ECS": 3.9,
        "TCRE": 1.87,
        "TCR": 1.9,
        "R2x": 2.86,
        "stop_1000": 68,
        "stop_750": 54,
        "stop_2000": 116,
        "efficacy": 1.24,
        "tas_pi": 287.6,
    },
    "BERN-ecs2k": {
        "ECS": 1.65,
        "TCRE": 1.19,
        "TCR": 1.22,
        "R2x": 3.71,
        "stop_1000": 1918.5,
        "stop_750": 1905.5,
        "stop_2000": 1959.5,
        "efficacy": 1.01,
        "tas_pi": 287.47,
    },
    "BERN-ecs3k": {
        "ECS": 2.57,
        "TCRE": 1.54,
        "TCR": 1.58,
        "R2x": 3.71,
        "stop_1000": 1918.5,
        "stop_750": 1905.5,
        "stop_2000": 1960.5,
        "efficacy": .95,
        "tas_pi": 287.6,
    },
    "BERN-ecs5k": {
        "ECS": 5.27,
        "TCRE": 2.22,
        "TCR": 2.17,
        "R2x": 3.71,
        "stop_1000": 1919.5,
        "stop_750": 1906.5,
        "stop_2000": 1962.5,
        "efficacy": 1.07,
        "tas_pi": 287.675
    },
    "CanESM5": {
        "ECS": 5.7,
        "TCRE": 2.27,
        "TCR": 2.8,
        "R2x": 3.35,
        "stop_1000": 1911,
        "stop_2000": 1947,
        "efficacy": 1.04,
        "tas_pi": 13.28
    },
    "CESM": {
        "ECS": np.nan,
        "TCRE": 1.99,
        "TCR": 2.0,
        "R2x": np.nan,
        "stop_1000": 68,
        "efficacy": np.nan,
        "tas_pi": 287.15
    },
    "CNRM-ESM2-1": {
        "ECS": 4.55,
        "TCRE": 1.9,
        "TCR": 1.92,
        "R2x": 3.23,
        "stop_1000": 1920,
        "efficacy": .93,
        "tas_pi": 286.71
    },
    "CLIMBER2": {
        "ECS": 2.8,
        "TCRE": 1.71,
        "TCR": 1.83,
        "R2x": 3.71,
        "stop_1000": 1917,
        "efficacy": None,
        "tas_pi": 287.12
    },
    "DCESS": {
        "ECS": 3.11,
        "TCRE": 2.02,
        "TCR": 2.02,
        "R2x": 3.71,
        "stop_1000": 1833.65,
        "stop_750": 1820.40,
        "stop_2000": 1874.40,
        "efficacy": 1.10,
        "tas_pi": 15.08
    },
    "GFDL-ESM2M": {
        "ECS": 2.4,
        "TCRE": 1.2,
        "TCR": 1.4,
        "R2x": 3.55,
        "stop_1000": 1922,
        "stop_750": 1910,
        "stop_2000": 1961,       
        "efficacy": 1.27,
        "tas_pi": 286.93948 
    },
    'IAPRAS': {
        'ECS': 2.18,
        "TCRE": 1.45,
        'TCR': 1.5,
        'R2x': 3.7,
        "stop_1000": 69,
        "stop_750": 56,
        "stop_2000": 109,   
        'efficacy': 1.13,
        "tas_pi": 286.4
    },
    "LOVECLIM": {
        "ECS": 2.8,
        "TCRE": 1.53,
        "TCR": 1.53,
        "R2x": 3.71,
        "stop_1000": 1920,
        "stop_750": 1905,
        "efficacy": .99,
        "tas_pi": 16.03 
    },
    "MESM": {
        "ECS": 2.87,
        "TCRE": 1.73,
        "TCR": 1.78,
        "R2x": 4.11,
        "stop_1000": 69,
        "stop_750": 57,
        "stop_2000": 111,  
        "efficacy": .79,
        "tas_pi": 13.30
    },
    'MIROC-ES2L': {
        'ECS': 2.7,
        "TCRE": 1.3,
        'TCR': 1.5,
        'R2x': 4.05,
        "stop_1000": 1913,
        "stop_750": 1901,
        "stop_2000": 1954, 
        "efficacy": .95,
        "tas_pi": 288.14
    },
    'MIROC-lite': {
        'ECS': 1.74,
        "TCRE": 1.06,
        'TCR': 1.16,
        'R2x': 2.97,
        "stop_1000": 66,
        "stop_750": 54,
        "stop_2000": 106, 
        "efficacy": .98,
        "tas_pi": 285.078
    },
    'MPIESM': {
        'ECS': 2.83,
        "TCRE": 1.63,
        'TCR': 1.82,
        'R2x': 4.1,
        "stop_1000": 1915,
        "stop_750": 67,
        "efficacy": 1.10,
        "tas_pi": 286.715
    },
    'NorESM2': {
        'ECS': 3.1,
        "TCRE": 1.4,
        'TCR': 1.48,
        'R2x': 2.4,
        "stop_1000": 67,
        "efficacy": 1.17,
        "tas_pi": 287.62
    },
    'PLASIM-GENIE': {
        'ECS': 3.4,
        "TCRE": 1.56,
        'TCR': 1.7,
        'R2x': 4.2,
        "stop_1000": 64,
        "stop_750": 51,
        "stop_2000": 104,     
        "efficacy": .9,
        "tas_pi": 286.8
    },
    'UKESM1': {
        'ECS': 5.4,
        "TCRE": 2.575,
        'TCR': 2.765,
        'R2x': 4.02,
        "stop_1000": 1917,
        "stop_750": 1904,
        "stop_2000": 1962,     
        'efficacy': 1.0,
        "tas_pi": 286.5
    },
    'UVic': {
        'ECS': 3.7,
        "TCRE": 1.82,
        'TCR': 1.87,
        'R2x': 4.07,
        "stop_1000": 1918,
        "stop_750": 1905,
        "stop_2000": 1963,     
        'efficacy': 1.01,
        "tas_pi": 286.59
    },
}

output = {}
for model in data:
    output[model] = {}
    output[model].update(metrics[model])
    

In [26]:
def process_model_exp(model, exp, data, output):
    """
    Process the data for a given model and experiment.

    Parameters:
    model (str): The model name.
    exp (str): The experiment identifier.
    data (dict): The data dictionary containing model data.
    output (dict): The output dictionary to store results.
    """
    if exp not in data[model]:
        return

    d = data[model][exp]
    output[model][exp] = {}
    o = output[model][exp]

    # Extract and process CO2 and temperature anomaly data
    o["co2"] = d["co2"].dropna().values
    o["tas"] = d["tas"].dropna().values
    o['delta_t'] = o["tas"] - output[model]["tas_pi"]
    o['delta_t_avg'] = climate_average(o['delta_t'])
    o['delta_co2'] = o["co2"] - o["co2"][0]

    # Compute stop index based on time
    stop_index = np.argmin(np.abs(d['time'] - output[model]['stop_' + exp]))
    o['stop_index'] = float(stop_index)

    # Compute airborne fraction
    o['airborne'] = 2.14 * o['delta_co2'] / float(exp)

    # Create an index array
    o['index'] = np.arange(len(o["tas"]))

    # Extract other variables
    o['cLand'] = d['cLand'].dropna().values
    o['fgco2'] = d['fgco2'].dropna().values

    # Optionally extract additional variables if available
    for key in ['nbp', 'rtmt', 'fco2nat']:
        if key in d:
            o[key if key != 'rtmt' else 'N'] = d[key].dropna().values

    # Compute radiative forcing
    o['Fo'] = output[model]['R2x'] / np.log(2)
    o['F'] = o['Fo'] * np.log(o['co2'] / o['co2'][0])

    # Compute final and zero-emission radiative forcing
    o['F_f'] = climate_average(o['F'])[-10]
    o['F_ze'] = climate_average(o['F'], tau=5)[int(o['stop_index'])]

    # Compute Zero Emissions Commitment (ZEC) average
    o['ZEC_avg'] = o['delta_t_avg'][o['index'] >= int(o['stop_index'])] - o['delta_t_avg'][int(o['stop_index'])]

    # Additional computations for certain models
    if model not in ["CESM", "PLASIM-GENIE", "BERN-ecs3k"] and 'N' in o:
        # Compute average net radiation at top of the atmosphere
        o["N_avg"] = climate_average(o['N'], 20)

        # Perform linear regression between net radiation and radiative forcing
        x = o['F'][:20]
        y = o['N'][:20]
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        output[model]['No'] = intercept

        # Compute change in net radiation
        o['dN'] = output[model]['efficacy'] * (o["N_avg"] - output[model]['No'])

        # Compute theoretical and experimental ZEC ratios
        o["ZEC_div_Tze_theory"] = (
            (o['F_f'] - o['dN'][-10]) / o['F_ze'] / (output[model]['TCR'] / output[model]['ECS']) - 1
        )
        o["ZEC_div_Tze_exp"] = o['ZEC_avg'][-10] / o['delta_t_avg'][int(o['stop_index'])]

# Main processing loop
for model in data:
    for exp in ["750", "1000", "2000"]:
        process_model_exp(model, exp, data, output)


In [27]:
#palette = sns.color_palette("tab20", 20) #cc.glasbey_dark
palette = cc.glasbey_dark
colors = {}
for name, color in zip(names, palette):
    output[name]['color'] = color

# convert data from array to lists for serialization
for model in data:
    for exp in ["750", "1000", "2000"]:
        if exp in output[model].keys():
            d = output[model][exp]
            for var in d.keys():
                if type(d[var]) is np.ndarray:
                    d[var] = d[var].tolist()
        
print("Writing out results to ZECMIP_data.json...")
with open(output_prefix + '/ZECMIP_data.json', 'w') as out:
    out.write(json.dumps(output))

Writing out results to ZECMIP_data.json...
